In [1]:
import mne
import os
from scipy import io as sio
import numpy as np

import mneflow
from mneflow import dev
#import tensorflow as tf





In [2]:
dpath = '/m/nbe/work/zubarei1/DLforMEG/continuous_ecog/fingers/'

if os.path.exists(dpath+'tfr/meta.pkl'):
    meta = mneflow.load_meta(dpath+'tfr/')
else:
    
    files = [fi for fi in os.listdir(dpath) if os.path.isfile(''.join([dpath, fi]))]
    fs = 2048;
    data_id = 'hse_ecog'
    decim=2

    #initialize metadata
    meta  = dict(train_paths=[],val_paths=[],orig_paths=[],
                 data_id=data_id,val_size=0, fs= None, 
                 savepath = dpath+'tfr/')

    for i,fname in enumerate(files):
        # load continuous data from matlab array
        data =  sio.loadmat(dpath+fname)
        X = np.transpose(data['ECOG']) #ecog
        Y = np.transpose(data['PN']) # kinematic signal
        #apply basic preprocessing
        X = mne.filter.notch_filter(X,fs,freqs=(50,100,150))
        X = mne.filter.filter_data(X,fs,l_freq=5,h_freq=333,method='iir')
        Y = mne.filter.filter_data(Y,fs,l_freq=.5,h_freq=5.,method='iir')
        X = X[:,::decim]
        Y = Y[:,::decim]

        #split into training and validation sets, divide each set into overlapping segments of 500 samples
        X_train, X_val, Y_train, Y_val = dev.preprocess_continuous([X,Y], val_size=.1, overlap=True, 
                                               segment=500, stride=25,scale=True)    
        del X, Y

        #update metadata file
        meta['n_ch'],meta['n_t'] = X_train.shape[1:]          
        meta['y_shape'] = Y_train.shape[1:]
        meta['task'] = 'ae'

        meta['train_paths'].append(''.join([meta['savepath'],meta['data_id'],'_train_',str(i),'.tfrecord']))
        meta['val_paths'].append(''.join([meta['savepath'],meta['data_id'],'_val_',str(i),'.tfrecord']))

        #wrtie TFRs
        mneflow.utils.write_tfrecords(X_train,Y_train,meta['train_paths'][-1], task='ae')
        mneflow.utils.write_tfrecords(X_val,Y_val,meta['val_paths'][-1], task='ae')



In [4]:
meta['savepath'] = dpath+'tfr/'
import numpy as np
optimizer_params = dict(l1_lambda=1e-12,
              learn_rate=3e-4,
              dropout = .5,
              patience = np.inf,# patientce for early stopping
              min_delta = 5e-6,
              n_iter = 30000, #total training epochs
              eval_step = 500, #evaluate validation loss each 10 epochs
              n_batch = 300,
              model_path = meta['savepath']
              ) #training batch size) 

ae_params = dict(n_ls=64, #number of latent factors
              filter_length=56, #convolutional filter length for var-cnn and lf-cnn
              pooling = 8, #convlayer pooling factor for var-cnn and lf-cnn
              stride = 8, #stride parameter for convolution filter
              df=20,
              padding = 'SAME'
              )

        

model = dev.VARDAE(meta, optimizer_params, ae_params)
model.build()


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Instructions for updating:
Colocations handled automatically by placer.
de-mix _init
x shape (?, 500, 1, 64)
conv _init
x shape (?, 500, 1, 64)
conv shape (?, 500, 1, 64)
conv  out shape: (?, 63, 1, 64)
x shape (?, 63, 1, 64)
conv _init
x shape (?, 63, 1, 64)
conv shape (?, 63, 1, 32)
conv  out shape: (?, 8, 1, 32)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
fc _init
deconv _init
x_reduced (?, 64)
x_perm (?, 1, 64)
deconv: (?, 500, 64)
(?, 10, 500)
y_pred: (?, 10, 500)
Initialization complete!


In [5]:
model.train()

i 0, tr_loss 1.49965e+06, tr_acc -0.0314735 v_loss 607417, v_acc -0.0309912


KeyboardInterrupt: 